<a href="https://colab.research.google.com/github/itsbonkk/IBM-granite/blob/main/%22timpa_teks%22_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain_community
!pip install -q replicate

In [ ]:
from langchain_community.llms import Replicate
import os
from google.colab import userdata
# Set the API token
api_token = userdata.get('api_token')
os.environ["REPLICATE_API_TOKEN"] = api_token
# Model setup
model = "ibm-granite/granite-3.3-8b-instruct"
output = Replicate(
model=model,
replicate_api_token=api_token,
)

In [165]:
#file foto dimasukkan ke dalam drive dan path tiap fotonya dimasukkan ke data_timpa1.csv
from google.colab import drive
drive.mount('/content/drive') # folder drive dimasukan ke colab

import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/project/LLM_timpa_teks/data_timpa1.csv',sep=";") #pandas untuk baca file csv

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [166]:
# cek info file csv : baris, kolom, dll
data.info()
print(data.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Nomer      64 non-null     int64 
 1   file_path  64 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.1+ KB
Index(['Nomer', 'file_path'], dtype='object')


In [167]:
!apt-get install -y tesseract-ocr
!pip install pytesseract pillow

import pytesseract #untuk OCR (Optical Character Recognition)
from PIL import Image

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [169]:
data["ocr_text"] = ""
# Looping setiap baris di DataFrame
for i, row in data.iterrows():
    img_path = row["file_path"]
    if os.path.exists(img_path): # Cek apakah file gambar ada
        text = pytesseract.image_to_string(Image.open(img_path))
        data.at[i, "ocr_text"] = text.strip()
    else:
        data.at[i, "ocr_text"] = "[FILE NOT FOUND]"  # Jika file tidak ada, diisi "[FILE NOT FOUND]"

In [170]:
import os

for path in data["file_path"]:
    print(path, os.path.exists(path))
barisError = data[data['ocr_text']== '[FILE NOT FOUND]']
print(barisError)

/content/drive/MyDrive/project/LLM_timpa_teks/doksli/doksli_1.jpeg True
/content/drive/MyDrive/project/LLM_timpa_teks/doksli/doksli_2.jpg True
/content/drive/MyDrive/project/LLM_timpa_teks/doksli/doksli_3.jpg True
/content/drive/MyDrive/project/LLM_timpa_teks/doksli/doksli_4.jpg True
/content/drive/MyDrive/project/LLM_timpa_teks/doksli/doksli_5.jpg True
/content/drive/MyDrive/project/LLM_timpa_teks/doksli/doksli_6.jpg True
/content/drive/MyDrive/project/LLM_timpa_teks/doksli/doksli_7.jpg True
/content/drive/MyDrive/project/LLM_timpa_teks/doksli/doksli_8.jpg True
/content/drive/MyDrive/project/LLM_timpa_teks/doksli/doksli_9.jpg True
/content/drive/MyDrive/project/LLM_timpa_teks/doksli/doksli_10.jpg True
/content/drive/MyDrive/project/LLM_timpa_teks/doksli/doksli_11.jpg True
/content/drive/MyDrive/project/LLM_timpa_teks/doksli/doksli_12.jpg True
/content/drive/MyDrive/project/LLM_timpa_teks/doksli/doksli_13.jpg True
/content/drive/MyDrive/project/LLM_timpa_teks/doksli/doksli_14.jpg True


In [171]:
parameters = {
    "temperature": 0.0,
    "top_k": 1,
    "top_p": 1.0,
    "max_tokens": 3,
    "min_tokens": 0,
    "random_seed": 42,
    "repetition_penalty": 1.0,
    "stopping_criteria": "length (3 tokens)",
    "stopping_sequence": ["\n"] }


In [ ]:
from langchain_core.prompts import PromptTemplate

# definisi prompt
template = """
Kamu bertugas mengklasifikasikan teks hasil OCR dari foto.

Definisi:
- "asli" = teks normal, konsisten, dan wajar. Misalnya berita instagram, pengumuman resmi, judul poster, atau teks dokumen. Termasuk judul berita meskipun clickbait.
- "meme" = teks berlebihan, konyol, mustahil, bercanda, atau parodi.
- "tidak terbaca" = teks acak, rusak, atau tidak bisa dikenali hasil OCR.

Aturan:
1. Jika teks kosong atau berisi karakter acak → jawab "tidak terbaca".
2. Jika teks adalah berita, pengumuman, atau dokumen resmi → jawab "asli".
3. Jika teks berlebihan, mustahil, atau bercanda → jawab "meme".
4. Output hanya 1 kata: "asli", "meme", atau "tidak terbaca". Jangan jelaskan apapun.
5. Jika ada teks yang bercampur antara terbaca dan tidak terbaca, hanya identifikasi yang terbaca saja.
6. jangan putuskan teks yang tidak terbaca selain ke "tidak terbaca"

Contoh:
Teks: "AKSI UNJUK RASA DI GEDUNG DPR RI"
Jawaban: asli

Teks: "Redbull resmi bakal pakai mesin L300 musim depan"
Jawaban: meme

Teks: "ASEAN Khawatir Indonesia Menjadi Negara dengan Hutang Terbanyak di 2030"
Jawaban: asli

Teks: "Kucing bisa jadi presiden kalau rajin cuci tangan"
Jawaban: meme

Sekarang klasifikasikan teks berikut: {{ teks_ocr }}
"""

prompt = PromptTemplate.from_template(template)

# label valid
label_valid = {"asli", "meme", "tidak terbaca"}

hasil = []

for i in range(0, 64):   # loop index 0--64
    baris = data.iloc[i]
    masukan = prompt.format(teks_ocr=baris["ocr_text"])
    jawaban = output.invoke(masukan)

    # normalisasi jawaban
    teks_jawaban = jawaban.strip().lower()

    # ambil label valid terakhir
    prediksi = None
    for token in teks_jawaban.split()[::-1]:
        if token in label_valid:
            prediksi = token
            break

    hasil.append({
        "indeks": i,
        "teks_ocr": baris["ocr_text"],
        "prediksi": prediksi
    })

# simpan ke DataFrame hasil
df_hasil = pd.DataFrame(hasil)

# tampilkan semua baris
pd.set_option("display.max_rows", None)
display(df_hasil)


,indeks,teks_ocr,prediksi
0,0,"ae\n\ntT\n\nCapai 1.500 Penonton,\nSutradara f...",meme
1,1,TR..NSTV\n\non\n\n \n\n~~\n\nUw Bede\nSIDANG...,meme
2,2,"ioe LIL NAS X,\n\nNGA\n\nan JALAN TANPA\nBUSAN...",meme
3,3,,asli
4,4,ta\n\nmel un)\n\nLedalatalL bG ma Leh dss\n\nA...,meme
5,5,——\n\n>\n———\n\nALPINE PAKAI\nMESIN MERCEDES\n...,asli
6,6,Tempo Media & coe XX\nvs 39 mnt-@\n\nHadapi De...,asli
7,7,f =“\n\n \n\nANTONY STARR UCAPKAN SELAMAT\nTIN...,asli
8,8,Korupsi Wamenaker:\n\nMasyarakat Diperas Rp6 J...,meme
9,9,3:30 ro 40 Peat | =\n\noS ))\n\n \n\nPENDERITA...,meme
